___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Victor Arruda Niubó

#### Giovanni Pertence Cardoso dos Santos

#### Gustavo Schlieper Tessitore

#### Leonardo Cesari Costa
___

# Previsão da quantidade de views

___

## 1. INTRODUÇÃO

O objetivo do projeto é estimar a quantidade de visualizações que um vídeo terá com base no título, horário de publicação e as tags atribuidas aos vídeos.

___
## 2. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [1]:
import pandas as pd
import numpy as np
import itertools
import json
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gugat\Desktop\Insper DS 2018.1\2.1 semestre\Dados\cdados_dp\P3


### BASE DE DADOS

In [3]:
# Base de dados dos videos em destaque do youtube US
you_data = pd.read_csv("USvideos.csv")
# jason que contém a relação id - categoria
ids = json.loads(open('US_category_id.json').read())

# filtro das variáveis de interesse
you_analysis = you_data.loc[you_data.views|you_data.title|you_data.tags|you_data.category_id,\
                           ['views', 'title', 'tags', 'category_id']]

you_analysis.head()

,views,title,tags,category_id
0,748374,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,22
1,2418783,The Trump Presidency: Last Week Tonight with J...,last week tonight trump presidency|last week t...,24
2,3191434,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman|rudy|mancuso|king|bach|racist|...,23
3,343168,Nickelback Lyrics: Real or Fake?,rhett and link|gmm|good mythical morning|rhett...,24
4,2095731,I Dare You: GOING BALD!?,ryan|higa|higatv|nigahiga|i dare you|idy|rhpc|...,24


In [4]:
# separação da string das tags em uma lista de strings
tags_split = you_analysis.tags.str.split(pat = '|')
you_analysis['tags_split'] = tags_split
you_analysis.head(3)

,views,title,tags,category_id,tags_split
0,748374,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,22,[SHANtell martin]
1,2418783,The Trump Presidency: Last Week Tonight with J...,last week tonight trump presidency|last week t...,24,"[last week tonight trump presidency, last week..."
2,3191434,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman|rudy|mancuso|king|bach|racist|...,23,"[racist superman, rudy, mancuso, king, bach, r..."


### ANÁLISE DESCRITIVA

In [91]:
merged = list(itertools.chain(*you_analysis.tags_split))
contador = pd.DataFrame({'col':merged})
limpino = contador[contador.col != '[none]']
limpino.col.value_counts()

funny                             3603
comedy                            2931
how to                            1604
music                             1302
Pop                               1280
2018                              1275
humor                             1185
food                              1159
science                           1111
review                            1005
makeup                             990
news                               988
celebrity                          930
vlog                               928
video                              890
tutorial                           864
live                               862
comedian                           861
interview                          845
cooking                            802
television                         779
celebrities                        769
fun                                737
movie                              734
cute                               730
family friendly          

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS